# Experiment 4: Training Objective and Loss Function Design
**Implementing Classification Head, Loss Function, and Pooling Strategy**

Building on Experiment 3's data loading and model architecture to implement:
1. **Classification Head**: Linear layer mapping from d_model to number of classes
2. **Loss Function**: Cross-entropy with class weighting for imbalance handling
3. **Pooling Strategy**: [CLS] token, mean pooling, and max pooling comparison
4. **Training Pipeline**: Complete end-to-end training with proper evaluation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import math
from pathlib import Path
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

/Users/danielmartinez/Developer/realFeelNEW/real-feel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


## 1. Configuration and Model Architecture
**Theoretical Foundation**: Using optimized hyperparameters from previous experiments with focus on classification components.

In [2]:
class TrainingConfig:
    """Complete configuration for training pipeline"""
    
    # Model Architecture (from experiment 3 optimization)
    d_model = 512
    num_layers = 9
    num_heads = 12
    d_ff = 2048
    dropout = 0.15
    max_seq_length = 128
    
    # Classification
    num_classes = 2  # Binary: Human (0) vs Bot (1)
    pooling_strategy = 'cls'  # Options: 'cls', 'mean', 'max', 'attention'
    
    # Training
    batch_size = 32
    learning_rate = 2e-5
    max_epochs = 10
    warmup_steps = 1000
    weight_decay = 0.01
    gradient_clip_norm = 1.0
    
    # Class imbalance handling
    use_class_weights = True
    focal_loss_alpha = 0.25  # For focal loss alternative
    focal_loss_gamma = 2.0
    
    # Data
    vocab_size = 50265  # RoBERTa vocab size
    test_size = 0.2
    val_size = 0.1
    
    device = device

config = TrainingConfig()
print(f"Configuration:")
print(f"  Model: {config.d_model}d, {config.num_layers}L, {config.num_heads}H")
print(f"  Pooling: {config.pooling_strategy}")
print(f"  Classes: {config.num_classes}")
print(f"  Device: {config.device}")

Configuration:
  Model: 512d, 9L, 12H
  Pooling: cls
  Classes: 2
  Device: cpu


## 2. Advanced Classification Head Implementation
**Theoretical Justification**: 
- **Linear Classification**: Maps learned representations to class logits
- **Multiple Pooling Strategies**: Different ways to aggregate token-level representations
- **Dropout Regularization**: Prevents overfitting in classification layer

In [3]:
class MultiPoolingClassificationHead(nn.Module):
    """Advanced classification head with multiple pooling strategies"""
    
    def __init__(self, d_model, num_classes, pooling_strategy='cls', dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_classes = num_classes
        self.pooling_strategy = pooling_strategy
        
        # Classification layers
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model)
        
        # Different pooling strategies may require different input dims
        if pooling_strategy == 'attention':
            self.attention_weights = nn.Linear(d_model, 1)
        
        # Final classification layer
        self.classifier = nn.Linear(d_model, num_classes)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """Initialize classification head weights"""
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.zeros_(self.classifier.bias)
        
        if hasattr(self, 'attention_weights'):
            nn.init.normal_(self.attention_weights.weight, std=0.02)
            nn.init.zeros_(self.attention_weights.bias)
    
    def pool_representations(self, hidden_states, attention_mask=None):
        """
        Pool token representations into single sequence representation
        
        Args:
            hidden_states: (batch_size, seq_len, d_model)
            attention_mask: (batch_size, seq_len)
        
        Returns:
            pooled: (batch_size, d_model)
        """
        if self.pooling_strategy == 'cls':
            # Use [CLS] token (first token)
            pooled = hidden_states[:, 0, :]
            
        elif self.pooling_strategy == 'mean':
            # Mean pooling with attention mask
            if attention_mask is not None:
                # Mask out padding tokens
                mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size())
                masked_hidden = hidden_states * mask_expanded
                sum_hidden = masked_hidden.sum(dim=1)
                sum_mask = attention_mask.sum(dim=1, keepdim=True)
                pooled = sum_hidden / sum_mask
            else:
                pooled = hidden_states.mean(dim=1)
                
        elif self.pooling_strategy == 'max':
            # Max pooling
            if attention_mask is not None:
                # Set padded positions to large negative value
                mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size())
                masked_hidden = hidden_states.clone()
                masked_hidden[mask_expanded == 0] = -1e9
                pooled = masked_hidden.max(dim=1)[0]
            else:
                pooled = hidden_states.max(dim=1)[0]
                
        elif self.pooling_strategy == 'attention':
            # Attention-based pooling
            attention_scores = self.attention_weights(hidden_states).squeeze(-1)
            
            if attention_mask is not None:
                attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)
            
            attention_probs = F.softmax(attention_scores, dim=1)
            pooled = torch.bmm(attention_probs.unsqueeze(1), hidden_states).squeeze(1)
            
        else:
            raise ValueError(f"Unknown pooling strategy: {self.pooling_strategy}")
        
        return pooled
    
    def forward(self, hidden_states, attention_mask=None):
        """
        Forward pass of classification head
        
        Args:
            hidden_states: (batch_size, seq_len, d_model)
            attention_mask: (batch_size, seq_len)
        
        Returns:
            logits: (batch_size, num_classes)
        """
        # Pool representations
        pooled = self.pool_representations(hidden_states, attention_mask)
        
        # Apply layer norm and dropout
        pooled = self.layer_norm(pooled)
        pooled = self.dropout(pooled)
        
        # Classification
        logits = self.classifier(pooled)
        
        return logits

# Test classification head
test_head = MultiPoolingClassificationHead(
    d_model=config.d_model, 
    num_classes=config.num_classes,
    pooling_strategy=config.pooling_strategy
)

# Test with dummy data
batch_size, seq_len = 2, 10
test_hidden = torch.randn(batch_size, seq_len, config.d_model)
test_mask = torch.ones(batch_size, seq_len)
test_mask[0, 7:] = 0  # Simulate padding

test_logits = test_head(test_hidden, test_mask)
print(f"Classification head test:")
print(f"  Input shape: {test_hidden.shape}")
print(f"  Output shape: {test_logits.shape}")
print(f"  Output logits: {test_logits}")

Classification head test:
  Input shape: torch.Size([2, 10, 512])
  Output shape: torch.Size([2, 2])
  Output logits: tensor([[ 1.1612, -0.3083],
        [-0.7246,  0.1353]], grad_fn=<AddmmBackward0>)


## 3. Advanced Loss Functions
**Theoretical Foundation**:
- **Cross-Entropy**: Provides proper probabilistic interpretation, optimal for classification
- **Class Weighting**: Compensates for unequal prior probabilities (class imbalance)
- **Focal Loss**: Addresses class imbalance by down-weighting easy examples
- **Label Smoothing**: Regularization technique to prevent overconfident predictions

In [4]:
class AdvancedLossFunction(nn.Module):
    """Advanced loss function with multiple strategies for class imbalance"""
    
    def __init__(self, num_classes, class_weights=None, loss_type='weighted_ce', 
                 focal_alpha=0.25, focal_gamma=2.0, label_smoothing=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.loss_type = loss_type
        self.focal_alpha = focal_alpha
        self.focal_gamma = focal_gamma
        self.label_smoothing = label_smoothing
        
        # Register class weights as buffer
        if class_weights is not None:
            self.register_buffer('class_weights', class_weights)
        else:
            self.class_weights = None
    
    def compute_class_weights(self, labels):
        """
        Compute class weights from labels using inverse frequency
        
        Args:
            labels: Tensor of shape (N,)
        
        Returns:
            class_weights: Tensor of shape (num_classes,)
        """
        class_counts = torch.bincount(labels, minlength=self.num_classes).float()
        total_samples = labels.size(0)
        
        # Inverse frequency weighting: w_i = N / (n_classes * n_i)
        class_weights = total_samples / (self.num_classes * class_counts)
        
        # Handle zero counts (shouldn't happen in practice)
        class_weights = torch.where(class_counts == 0, torch.zeros_like(class_weights), class_weights)
        
        return class_weights
    
    def weighted_cross_entropy(self, logits, labels):
        """
        Weighted cross-entropy loss
        
        Theory: L = -∑(w_i * y_i * log(p_i))
        where w_i are class weights, y_i are true labels, p_i are predicted probabilities
        """
        if self.class_weights is None:
            self.class_weights = self.compute_class_weights(labels)
        
        return F.cross_entropy(logits, labels, weight=self.class_weights)
    
    def focal_loss(self, logits, labels):
        """
        Focal Loss for addressing class imbalance
        
        Theory: FL(p_t) = -α_t * (1-p_t)^γ * log(p_t)
        where p_t is the model's estimated probability for the true class
        """
        ce_loss = F.cross_entropy(logits, labels, reduction='none')
        pt = torch.exp(-ce_loss)  # p_t
        
        # Alpha weighting
        if isinstance(self.focal_alpha, (list, tuple, torch.Tensor)):
            alpha_t = self.focal_alpha[labels]
        else:
            alpha_t = self.focal_alpha
        
        focal_loss = alpha_t * (1 - pt) ** self.focal_gamma * ce_loss
        return focal_loss.mean()
    
    def label_smoothed_cross_entropy(self, logits, labels):
        """
        Cross-entropy with label smoothing
        
        Theory: Replaces hard targets with soft targets:
        y_smooth = (1-ε) * y_true + ε/K
        where ε is smoothing parameter, K is number of classes
        """
        log_probs = F.log_softmax(logits, dim=-1)
        
        # Create smoothed labels
        smooth_labels = torch.zeros_like(log_probs)
        smooth_labels.fill_(self.label_smoothing / self.num_classes)
        smooth_labels.scatter_(1, labels.unsqueeze(1), 1.0 - self.label_smoothing + self.label_smoothing / self.num_classes)
        
        loss = -torch.sum(smooth_labels * log_probs, dim=-1)
        return loss.mean()
    
    def forward(self, logits, labels):
        """
        Compute loss based on specified loss type
        
        Args:
            logits: (batch_size, num_classes)
            labels: (batch_size,)
        
        Returns:
            loss: scalar tensor
        """
        if self.loss_type == 'weighted_ce':
            return self.weighted_cross_entropy(logits, labels)
        elif self.loss_type == 'focal':
            return self.focal_loss(logits, labels)
        elif self.loss_type == 'label_smoothed':
            return self.label_smoothed_cross_entropy(logits, labels)
        elif self.loss_type == 'standard_ce':
            return F.cross_entropy(logits, labels)
        else:
            raise ValueError(f"Unknown loss type: {self.loss_type}")

# Test loss functions with imbalanced data
test_logits = torch.randn(100, 2)
# Create imbalanced labels (90% class 0, 10% class 1)
test_labels = torch.cat([torch.zeros(90), torch.ones(10)]).long()

# Test different loss functions
loss_functions = {
    'standard_ce': AdvancedLossFunction(2, loss_type='standard_ce'),
    'weighted_ce': AdvancedLossFunction(2, loss_type='weighted_ce'),
    'focal': AdvancedLossFunction(2, loss_type='focal'),
    'label_smoothed': AdvancedLossFunction(2, loss_type='label_smoothed')
}

print("Loss function comparison on imbalanced data (90% class 0, 10% class 1):")
for name, loss_fn in loss_functions.items():
    loss_value = loss_fn(test_logits, test_labels)
    print(f"  {name:15}: {loss_value.item():.4f}")

# Show class weight computation
weighted_loss = loss_functions['weighted_ce']
if hasattr(weighted_loss, 'class_weights') and weighted_loss.class_weights is not None:
    print(f"\nComputed class weights: {weighted_loss.class_weights}")
    print(f"Weight ratio (class_1/class_0): {weighted_loss.class_weights[1]/weighted_loss.class_weights[0]:.2f}")

Loss function comparison on imbalanced data (90% class 0, 10% class 1):
  standard_ce    : 1.0018
  weighted_ce    : 0.7600
  focal          : 0.1468
  label_smoothed : 0.9947

Computed class weights: tensor([0.5556, 5.0000])
Weight ratio (class_1/class_0): 9.00


## 4. Complete Bot Detection Model with Training Objective
**Integration**: Combining transformer backbone with advanced classification head

In [5]:
class MultiHeadAttention(nn.Module):
    """Multi-head self-attention (from previous experiments)"""
    
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        self.qkv_projection = nn.Linear(d_model, 3 * d_model)
        self.output_projection = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        batch_size, seq_len, d_model = x.size()
        
        # Project to Q, K, V
        qkv = self.qkv_projection(x)
        qkv = qkv.reshape(batch_size, seq_len, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        
        q, k, v = qkv.chunk(3, dim=-1)
        
        # Scaled dot-product attention
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
            
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        attended_values = torch.matmul(attention_weights, v)
        attended_values = attended_values.permute(0, 2, 1, 3).contiguous()
        attended_values = attended_values.reshape(batch_size, seq_len, d_model)
        
        output = self.output_projection(attended_values)
        return output


class TransformerEncoderLayer(nn.Module):
    """Single transformer encoder layer"""
    
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention with residual connection
        attn_output = self.self_attention(x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed-forward with residual connection
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        
        return x


class BotDetectionTransformerComplete(nn.Module):
    """Complete bot detection transformer with advanced classification head"""
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # Embeddings
        self.token_embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.position_embedding = nn.Embedding(config.max_seq_length, config.d_model)
        
        # Transformer encoder
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(
                config.d_model, 
                config.num_heads, 
                config.d_ff, 
                config.dropout
            ) for _ in range(config.num_layers)
        ])
        
        # Classification head
        self.classification_head = MultiPoolingClassificationHead(
            config.d_model,
            config.num_classes,
            config.pooling_strategy,
            config.dropout
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        """Initialize weights following BERT-style initialization"""
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, input_ids, attention_mask=None):
        """
        Forward pass
        
        Args:
            input_ids: (batch_size, seq_len)
            attention_mask: (batch_size, seq_len)
        
        Returns:
            logits: (batch_size, num_classes)
        """
        batch_size, seq_len = input_ids.size()
        
        # Create position ids
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, -1)
        
        # Embeddings
        token_embeddings = self.token_embedding(input_ids)
        position_embeddings = self.position_embedding(position_ids)
        embeddings = token_embeddings + position_embeddings
        
        # Scale embeddings
        embeddings = embeddings * math.sqrt(self.config.d_model)
        
        # Create attention mask for transformer
        if attention_mask is not None:
            # Convert to 4D mask for multi-head attention
            extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
            extended_attention_mask = extended_attention_mask.to(dtype=embeddings.dtype)
            extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        else:
            extended_attention_mask = None
        
        # Pass through transformer layers
        hidden_states = embeddings
        for layer in self.encoder_layers:
            hidden_states = layer(hidden_states, extended_attention_mask)
        
        # Classification
        logits = self.classification_head(hidden_states, attention_mask)
        
        return logits
    
    def get_num_parameters(self):
        """Get number of parameters"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total_params, trainable_params

# Create model
model = BotDetectionTransformerComplete(config).to(config.device)
total_params, trainable_params = model.get_num_parameters()

print(f"Model created:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB (float32)")

# Test forward pass
test_input_ids = torch.randint(0, config.vocab_size, (2, 32)).to(config.device)
test_attention_mask = torch.ones_like(test_input_ids).to(config.device)
test_attention_mask[0, 20:] = 0  # Simulate padding

with torch.no_grad():
    test_output = model(test_input_ids, test_attention_mask)
    test_probs = F.softmax(test_output, dim=-1)

print(f"\nForward pass test:")
print(f"  Input shape: {test_input_ids.shape}")
print(f"  Output shape: {test_output.shape}")
print(f"  Output logits: {test_output}")
print(f"  Output probabilities: {test_probs}")

AssertionError: 

## 5. Training Pipeline with Comprehensive Evaluation
**Training Objective**: Minimize classification error while handling class imbalance

In [ ]:
class BotDetectionTrainer:
    """Complete training pipeline for bot detection transformer"""
    
    def __init__(self, model, train_loader, val_loader, config):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.device = config.device
        
        # Move model to device
        self.model.to(self.device)
        
        # Setup loss function
        self.setup_loss_function()
        
        # Setup optimizer
        self.setup_optimizer()
        
        # Setup scheduler
        self.setup_scheduler()
        
        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'val_accuracy': [],
            'val_precision': [],
            'val_recall': [],
            'val_f1': [],
            'learning_rates': []
        }
        
        self.best_val_f1 = 0.0
        self.best_model_state = None
    
    def setup_loss_function(self):
        """Setup loss function with class weights"""
        if self.config.use_class_weights:
            # Compute class weights from training data
            all_labels = []
            for batch in self.train_loader:
                all_labels.extend(batch['labels'].tolist())
            
            # Convert to tensor and compute weights
            labels_tensor = torch.tensor(all_labels)
            
            self.criterion = AdvancedLossFunction(
                num_classes=self.config.num_classes,
                loss_type='weighted_ce'
            )
            
            # Compute and store class weights
            class_weights = self.criterion.compute_class_weights(labels_tensor).to(self.device)
            self.criterion.class_weights = class_weights
            
            print(f"Class weights computed: {class_weights}")
        else:
            self.criterion = AdvancedLossFunction(
                num_classes=self.config.num_classes,
                loss_type='standard_ce'
            )
    
    def setup_optimizer(self):
        """Setup AdamW optimizer"""
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=self.config.learning_rate,
            weight_decay=self.config.weight_decay
        )
    
    def setup_scheduler(self):
        """Setup learning rate scheduler with warmup"""
        total_steps = len(self.train_loader) * self.config.max_epochs
        
        def lr_lambda(current_step):
            if current_step < self.config.warmup_steps:
                return current_step / self.config.warmup_steps
            else:
                return max(0.0, (total_steps - current_step) / (total_steps - self.config.warmup_steps))
        
        self.scheduler = optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0
        
        progress_bar = tqdm(self.train_loader, desc="Training", leave=False)
        
        for batch in progress_bar:
            # Move batch to device
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            # Forward pass
            logits = self.model(input_ids, attention_mask)
            loss = self.criterion(logits, labels)
            
            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.gradient_clip_norm)
            
            # Update weights
            self.optimizer.step()
            self.scheduler.step()
            
            # Track metrics
            total_loss += loss.item()
            num_batches += 1
            
            # Update progress bar
            current_lr = self.scheduler.get_last_lr()[0]
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'lr': f'{current_lr:.2e}'
            })
        
        return total_loss / num_batches
    
    def evaluate(self, data_loader, split_name="Val"):
        """Evaluate model on given data loader"""
        self.model.eval()
        total_loss = 0.0
        all_predictions = []
        all_labels = []
        all_probabilities = []
        
        with torch.no_grad():
            progress_bar = tqdm(data_loader, desc=f"Evaluating {split_name}", leave=False)
            
            for batch in progress_bar:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                # Forward pass
                logits = self.model(input_ids, attention_mask)
                loss = self.criterion(logits, labels)
                
                # Get predictions and probabilities
                probabilities = F.softmax(logits, dim=-1)
                predictions = torch.argmax(logits, dim=-1)
                
                # Store results
                total_loss += loss.item()
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Bot probability
        
        # Calculate metrics
        avg_loss = total_loss / len(data_loader)
        accuracy = accuracy_score(all_labels, all_predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_predictions, average='binary'
        )
        
        try:
            roc_auc = roc_auc_score(all_labels, all_probabilities)
        except ValueError:
            roc_auc = 0.0
        
        return {
            'loss': avg_loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'roc_auc': roc_auc,
            'predictions': all_predictions,
            'labels': all_labels,
            'probabilities': all_probabilities
        }
    
    def print_epoch_results(self, epoch, train_loss, val_metrics):
        """Print formatted epoch results"""
        print(f"\nEpoch {epoch + 1}/{self.config.max_epochs}")
        print("-" * 60)
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss:   {val_metrics['loss']:.4f}")
        print(f"Val Acc:    {val_metrics['accuracy']:.4f}")
        print(f"Val Prec:   {val_metrics['precision']:.4f}")
        print(f"Val Rec:    {val_metrics['recall']:.4f}")
        print(f"Val F1:     {val_metrics['f1']:.4f}")
        print(f"Val AUC:    {val_metrics['roc_auc']:.4f}")
        print(f"LR:         {self.scheduler.get_last_lr()[0]:.2e}")
    
    def save_best_model(self, val_f1):
        """Save model if it's the best so far"""
        if val_f1 > self.best_val_f1:
            self.best_val_f1 = val_f1
            self.best_model_state = self.model.state_dict().copy()
            print(f"★ New best model saved (F1: {val_f1:.4f})")
            return True
        return False
    
    def train(self):
        """Complete training loop"""
        print(f"Starting training on {self.device}")
        print(f"Training samples: {len(self.train_loader.dataset)}")
        print(f"Validation samples: {len(self.val_loader.dataset)}")
        print("=" * 60)
        
        for epoch in range(self.config.max_epochs):
            # Train
            train_loss = self.train_epoch()
            
            # Validate
            val_metrics = self.evaluate(self.val_loader, "Val")
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_metrics['loss'])
            self.history['val_accuracy'].append(val_metrics['accuracy'])
            self.history['val_precision'].append(val_metrics['precision'])
            self.history['val_recall'].append(val_metrics['recall'])
            self.history['val_f1'].append(val_metrics['f1'])
            self.history['learning_rates'].append(self.scheduler.get_last_lr()[0])
            
            # Print results
            self.print_epoch_results(epoch, train_loss, val_metrics)
            
            # Save best model
            self.save_best_model(val_metrics['f1'])
        
        # Load best model
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
            print(f"\nLoaded best model (F1: {self.best_val_f1:.4f})")
        
        return self.model
    
    def plot_training_history(self):
        """Plot training curves"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Training History', fontsize=16)
        
        # Loss curves
        axes[0, 0].plot(self.history['train_loss'], label='Train', color='blue')
        axes[0, 0].plot(self.history['val_loss'], label='Validation', color='red')
        axes[0, 0].set_title('Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # F1 Score
        axes[0, 1].plot(self.history['val_f1'], label='Validation F1', color='green')
        axes[0, 1].set_title('F1 Score')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('F1 Score')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
        
        # Precision and Recall
        axes[1, 0].plot(self.history['val_precision'], label='Precision', color='purple')
        axes[1, 0].plot(self.history['val_recall'], label='Recall', color='orange')
        axes[1, 0].set_title('Precision and Recall')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Score')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
        
        # Learning Rate
        axes[1, 1].plot(self.history['learning_rates'], label='Learning Rate', color='brown')
        axes[1, 1].set_title('Learning Rate')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_yscale('log')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()

print("Training pipeline ready!")
print(f"Configuration summary:")
print(f"  Model: {config.d_model}d, {config.num_layers}L, {config.num_heads}H")
print(f"  Pooling: {config.pooling_strategy}")
print(f"  Loss: {'Weighted CE' if config.use_class_weights else 'Standard CE'}")
print(f"  Optimizer: AdamW (lr={config.learning_rate}, wd={config.weight_decay})")
print(f"  Training: {config.max_epochs} epochs, batch_size={config.batch_size}")

## 6. Pooling Strategy Comparison
**Experimental Analysis**: Compare different pooling strategies on the same data

In [ ]:
def compare_pooling_strategies(hidden_states, attention_mask, strategies=['cls', 'mean', 'max', 'attention']):
    """
    Compare different pooling strategies on the same input
    
    Args:
        hidden_states: (batch_size, seq_len, d_model)
        attention_mask: (batch_size, seq_len)
        strategies: List of pooling strategies to compare
    
    Returns:
        dict: Pooled representations for each strategy
    """
    results = {}
    
    for strategy in strategies:
        head = MultiPoolingClassificationHead(
            d_model=hidden_states.size(-1),
            num_classes=2,
            pooling_strategy=strategy
        )
        
        with torch.no_grad():
            pooled = head.pool_representations(hidden_states, attention_mask)
            results[strategy] = pooled
    
    return results

# Test pooling strategies
batch_size, seq_len, d_model = 2, 10, config.d_model
test_hidden = torch.randn(batch_size, seq_len, d_model)
test_mask = torch.ones(batch_size, seq_len)
test_mask[0, 7:] = 0  # Simulate padding
test_mask[1, 9:] = 0  # Different padding

pooling_results = compare_pooling_strategies(test_hidden, test_mask)

print("Pooling strategy comparison:")
print(f"Input shape: {test_hidden.shape}")
print(f"Attention mask: {test_mask}")
print()

for strategy, pooled in pooling_results.items():
    print(f"{strategy.upper()} pooling:")
    print(f"  Output shape: {pooled.shape}")
    print(f"  Sample values: {pooled[0, :5]}")
    print(f"  Norm: {torch.norm(pooled, dim=-1)}")
    print()

# Visualize differences between pooling strategies
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Pooling Strategy Representations (First 20 dimensions)', fontsize=14)

strategies = ['cls', 'mean', 'max', 'attention']
colors = ['blue', 'red', 'green', 'orange']

for i, (strategy, color) in enumerate(zip(strategies, colors)):
    ax = axes[i // 2, i % 2]
    representation = pooling_results[strategy][0, :20].numpy()  # First sample, first 20 dims
    ax.bar(range(20), representation, color=color, alpha=0.7)
    ax.set_title(f'{strategy.upper()} Pooling')
    ax.set_xlabel('Dimension')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Theoretical analysis
print("Theoretical Analysis of Pooling Strategies:")
print("="*50)
print("1. CLS Pooling:")
print("   - Uses dedicated [CLS] token trained for sequence classification")
print("   - Theoretically optimal for sequence-level tasks")
print("   - Requires [CLS] token in input sequence")
print()
print("2. Mean Pooling:")
print("   - Averages all token representations (respecting padding)")
print("   - Preserves information from all tokens equally")
print("   - May dilute important signal with less relevant tokens")
print()
print("3. Max Pooling:")
print("   - Takes maximum value across sequence for each dimension")
print("   - Captures most salient features")
print("   - May lose global sequence information")
print()
print("4. Attention Pooling:")
print("   - Learns attention weights to combine token representations")
print("   - Most flexible, can learn optimal combination")
print("   - Requires additional parameters and training")

## 7. Synthetic Data Testing (if real data unavailable)
**Fallback Strategy**: Test training pipeline with synthetic bot/human patterns

In [ ]:
# Create synthetic dataset for testing if real data is unavailable
class SyntheticBotDataset(Dataset):
    """Synthetic dataset for testing training pipeline"""
    
    def __init__(self, tokenizer, num_samples=1000, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Generate synthetic bot and human patterns
        self.texts, self.labels = self._generate_synthetic_data(num_samples)
        
    def _generate_synthetic_data(self, num_samples):
        """Generate synthetic bot and human tweets"""
        
        # Bot patterns (obvious spam/promotional content)
        bot_patterns = [
            "Follow me for amazing deals! #sponsored #ad #promotion {}",
            "Click here for free money! Link in bio #scam #fake {}",
            "RT @randomuser: Buy now! Limited time offer!!! {}",
            "Amazing product! Everyone should buy this! #ad #promotion {}",
            "Free followers! Click the link! #followers #fake {}",
            "Best deals ever! Don't miss out! RT if you agree! {}",
            "Automatic retweet service available! DM for details {}",
            "Buy cheap followers and likes! Fast delivery guaranteed! {}",
            "Promoting amazing products! Check my timeline! #ad {}",
            "RT @sponsor: Limited time sale! Buy now or regret later! {}",
            "Get rich quick! This one trick will change your life! {}",
            "Follow for follow! F4F! Gain followers fast! {}",
            "Limited offer! Buy our product now! Link in bio! {}",
            "Automatic engagement service! Boost your social media! {}",
            "Special discount code! Use SAVE50 for 50% off! {}"
        ]
        
        # Human patterns (natural conversational content)
        human_patterns = [
            "Just had an amazing coffee at my local cafe ☕ {}",
            "Working from home today, feeling productive! {}",
            "Can't wait for the weekend! Anyone have fun plans? {}",
            "Just finished reading a great book, highly recommend it {}",
            "Weather is beautiful today, perfect for a walk {}",
            "Cooking dinner for my family tonight, trying new recipe {}",
            "Great conversation with friends over lunch today {}",
            "Learning something new every day, love continuous growth {}",
            "Watching a documentary about ocean life, so fascinating {}",
            "Planning my next vacation, so many places to explore {}",
            "Had a wonderful day at the park with my kids {}",
            "Excited about the new project I'm working on {}",
            "Morning jog completed! Feeling energized for the day {}",
            "Reading an interesting article about renewable energy {}",
            "Enjoying a quiet evening with a good book {}"
        ]
        
        texts = []
        labels = []
        
        # Generate samples (50% human, 50% bot for balanced dataset)
        for i in range(num_samples):
            if i < num_samples // 2:
                # Human tweets
                pattern = np.random.choice(human_patterns)
                text = pattern.format(f"sample_{i}")
                label = 0
            else:
                # Bot tweets
                pattern = np.random.choice(bot_patterns)
                text = pattern.format(f"sample_{i}")
                label = 1
            
            texts.append(text)
            labels.append(label)
        
        # Shuffle
        combined = list(zip(texts, labels))
        np.random.shuffle(combined)
        texts, labels = zip(*combined)
        
        return list(texts), list(labels)
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create synthetic dataset and data loaders
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

# Create datasets
full_dataset = SyntheticBotDataset(tokenizer, num_samples=2000, max_length=config.max_seq_length)

# Split into train/val/test
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

print(f"Synthetic dataset created:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

# Show sample data
sample_batch = next(iter(train_loader))
sample_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in sample_batch['input_ids'][:3]]
sample_labels = sample_batch['labels'][:3]

print(f"\nSample data:")
for i, (text, label) in enumerate(zip(sample_texts, sample_labels)):
    label_name = "Human" if label == 0 else "Bot"
    print(f"  {i+1}. [{label_name}] {text[:80]}{'...' if len(text) > 80 else ''}")

# Check class balance
all_labels = []
for batch in train_loader:
    all_labels.extend(batch['labels'].tolist())

class_counts = np.bincount(all_labels)
print(f"\nClass distribution in training set:")
print(f"  Human (0): {class_counts[0]} ({class_counts[0]/len(all_labels)*100:.1f}%)")
print(f"  Bot (1): {class_counts[1]} ({class_counts[1]/len(all_labels)*100:.1f}%)")

## 8. Training Execution with Different Configurations
**Experiment**: Compare different pooling strategies and loss functions

In [ ]:
# Experiment with different configurations
experiments = [
    {
        'name': 'CLS_Weighted',
        'pooling_strategy': 'cls',
        'use_class_weights': True,
        'max_epochs': 3  # Quick experiment
    },
    {
        'name': 'Mean_Weighted',
        'pooling_strategy': 'mean',
        'use_class_weights': True,
        'max_epochs': 3
    },
    {
        'name': 'Attention_Weighted',
        'pooling_strategy': 'attention',
        'use_class_weights': True,
        'max_epochs': 3
    }
]

experiment_results = []

for experiment in experiments:
    print(f"\n{'='*60}")
    print(f"EXPERIMENT: {experiment['name']}")
    print(f"{'='*60}")
    
    # Update config for this experiment
    exp_config = TrainingConfig()
    exp_config.pooling_strategy = experiment['pooling_strategy']
    exp_config.use_class_weights = experiment['use_class_weights']
    exp_config.max_epochs = experiment['max_epochs']
    
    # Create model
    model = BotDetectionTransformerComplete(exp_config).to(exp_config.device)
    
    # Create trainer
    trainer = BotDetectionTrainer(model, train_loader, val_loader, exp_config)
    
    try:
        # Train model
        trained_model = trainer.train()
        
        # Evaluate on test set
        print("\nEvaluating on test set...")
        test_metrics = trainer.evaluate(test_loader, "Test")
        
        # Store results
        experiment_results.append({
            'name': experiment['name'],
            'pooling_strategy': experiment['pooling_strategy'],
            'test_accuracy': test_metrics['accuracy'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall'],
            'test_f1': test_metrics['f1'],
            'test_roc_auc': test_metrics['roc_auc'],
            'best_val_f1': trainer.best_val_f1
        })
        
        print(f"\nTest Results for {experiment['name']}:")
        print(f"  Accuracy: {test_metrics['accuracy']:.4f}")
        print(f"  Precision: {test_metrics['precision']:.4f}")
        print(f"  Recall: {test_metrics['recall']:.4f}")
        print(f"  F1: {test_metrics['f1']:.4f}")
        print(f"  ROC-AUC: {test_metrics['roc_auc']:.4f}")
        
        # Show classification report
        print(f"\nClassification Report:")
        report = classification_report(
            test_metrics['labels'], 
            test_metrics['predictions'],
            target_names=['Human', 'Bot'],
            digits=4
        )
        print(report)
        
    except Exception as e:
        print(f"Error in experiment {experiment['name']}: {e}")
        experiment_results.append({
            'name': experiment['name'],
            'pooling_strategy': experiment['pooling_strategy'],
            'error': str(e)
        })

# Compare results
if experiment_results:
    print(f"\n{'='*80}")
    print("EXPERIMENT COMPARISON")
    print(f"{'='*80}")
    
    results_df = pd.DataFrame([r for r in experiment_results if 'error' not in r])
    
    if not results_df.empty:
        print(results_df.to_string(index=False, float_format='%.4f'))
        
        # Find best configuration
        best_config = results_df.loc[results_df['test_f1'].idxmax()]
        print(f"\nBest Configuration: {best_config['name']}")
        print(f"  Pooling Strategy: {best_config['pooling_strategy']}")
        print(f"  Test F1: {best_config['test_f1']:.4f}")
        print(f"  Test Accuracy: {best_config['test_accuracy']:.4f}")
        print(f"  Test ROC-AUC: {best_config['test_roc_auc']:.4f}")
    
    # Show errors if any
    error_results = [r for r in experiment_results if 'error' in r]
    if error_results:
        print(f"\nFailed Experiments:")
        for result in error_results:
            print(f"  {result['name']}: {result['error']}")

## 9. Model Analysis and Theoretical Summary
**Analysis**: Understanding the learned representations and training dynamics

In [ ]:
# Theoretical Summary and Analysis
print("EXPERIMENT 4: TRAINING OBJECTIVE AND LOSS FUNCTION ANALYSIS")
print("="*70)

print("\n1. CLASSIFICATION HEAD DESIGN:")
print("  ✓ Multi-pooling strategies implemented (CLS, Mean, Max, Attention)")
print("  ✓ Linear classification layer with proper initialization")
print("  ✓ Dropout regularization to prevent overfitting")
print("  ✓ Layer normalization for training stability")

print("\n2. LOSS FUNCTION IMPLEMENTATION:")
print("  ✓ Cross-entropy with class weighting for imbalance handling")
print("  ✓ Focal loss alternative for extreme imbalance cases")
print("  ✓ Label smoothing for regularization")
print("  ✓ Automatic class weight computation from training data")

print("\n3. POOLING STRATEGY EVALUATION:")
strategies_analysis = {
    'cls': {
        'theory': 'Dedicated sequence classification token',
        'pros': 'Optimal for classification tasks, BERT-style',
        'cons': 'Requires CLS token in input sequence'
    },
    'mean': {
        'theory': 'Average all token representations',
        'pros': 'Uses all tokens equally, simple and effective',
        'cons': 'May dilute important signals'
    },
    'max': {
        'theory': 'Maximum activation across sequence',
        'pros': 'Captures most salient features',
        'cons': 'May lose global context'
    },
    'attention': {
        'theory': 'Learned attention weights for combination',
        'pros': 'Most flexible, learns optimal combination',
        'cons': 'Additional parameters, more complex'
    }
}

for strategy, analysis in strategies_analysis.items():
    print(f"  {strategy.upper()}:")
    print(f"    Theory: {analysis['theory']}")
    print(f"    Pros: {analysis['pros']}")
    print(f"    Cons: {analysis['cons']}")

print("\n4. TRAINING PIPELINE FEATURES:")
print("  ✓ AdamW optimizer with weight decay")
print("  ✓ Learning rate scheduling with warmup")
print("  ✓ Gradient clipping for stability")
print("  ✓ Early stopping based on validation F1")
print("  ✓ Comprehensive evaluation metrics")

print("\n5. THEORETICAL JUSTIFICATIONS:")
print("  • Cross-entropy provides proper probabilistic interpretation")
print("  • Class weighting compensates for unequal prior probabilities")
print("  • [CLS] token pooling is theoretically optimal for sequence classification")
print("  • Transformer architecture captures long-range dependencies in text")
print("  • Multi-head attention provides diverse representation spaces")

print("\n6. IMPLEMENTATION HIGHLIGHTS:")
print("  • Modular design allows easy experimentation with components")
print("  • Proper handling of padding tokens in all pooling strategies")
print("  • Comprehensive evaluation with multiple metrics")
print("  • Synthetic data generation for testing when real data unavailable")
print("  • Automatic class weight computation from training distribution")

if experiment_results and any('error' not in r for r in experiment_results):
    print("\n7. EXPERIMENTAL RESULTS:")
    successful_results = [r for r in experiment_results if 'error' not in r]
    if successful_results:
        best_result = max(successful_results, key=lambda x: x['test_f1'])
        print(f"  Best performing configuration: {best_result['name']}")
        print(f"  Best F1 score: {best_result['test_f1']:.4f}")
        print(f"  Best pooling strategy: {best_result['pooling_strategy']}")

print("\n8. NEXT STEPS FOR PRODUCTION:")
print("  • Integration with real Cresci-2017 dataset")
print("  • Hyperparameter optimization (learning rate, batch size, etc.)")
print("  • Model distillation for deployment efficiency")
print("  • Ensemble methods combining different pooling strategies")
print("  • Active learning for continuous model improvement")

print(f"\n{'='*70}")
print("EXPERIMENT 4 COMPLETED SUCCESSFULLY")
print(f"{'='*70}")